# Testing HermesPy

In [ ]:
import kcss
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.constants import speed_of_light
from hermespy.core import AntennaMode, ConsoleMode, Transformation
from hermespy.beamforming import BeamformingTransmitter, ConventionalBeamformer, SphericalFocus
from hermespy.simulation import SimulatedAntennaPort, SimulatedIdealAntenna, SimulatedCustomArray, Simulation


# Configure Antenna Array

The speed of light: 

$$c = 3 \times 10^{8} \space \text  m/s\ $$

The wavelength is $$\lambda=c / f$$

Antenna arrays are spread at half the wavelength


In [ ]:
# Parameters
sampling_rate = 1e9
carrier_frequency = 70e9
wavelength = speed_of_light / carrier_frequency

# =====================================
# Configure a grid antenna array
# =====================================
# Configures a 4x4 (16 element antenna array)
transmit_ports = [
    SimulatedAntennaPort(
        [SimulatedIdealAntenna(AntennaMode.TX)],
        Transformation.From_Translation(np.array([.5 * wavelength * x, .5 * wavelength * y, 0]))
    ) for x, y in product(range(4), range(4))
]

uniform_array = SimulatedCustomArray(transmit_ports)

# Visualize the antenna array element positions in the array's local coordinate system
tmp = uniform_array.plot_topology()
plt.show()

## Prepare a Signal

Prepare a sinusoid for transmission.



In [ ]:
# =====================================
# Create transmit signal (sinusoid)
# =====================================
from hermespy.core import Signal
sinusoid = Signal.Create(np.outer(np.ones(1), np.exp(2j * np.pi * .25 * np.arange(100))), sampling_rate, carrier_frequency)

# Configure the device to transmit a beamformed sinusoidal probing signal
beamformer = ConventionalBeamformer()
base_station_transmitter = BeamformingTransmitter(sinusoid, beamformer)

# =====================================
# Configure Base station 
# =====================================
# Initialize a new simulation
simulation = Simulation(console_mode=ConsoleMode.SILENT, seed=42)

# Create a new device and assign it the antenna array
base_station_device = simulation.new_device(
    antennas=uniform_array,
    carrier_frequency=carrier_frequency,
    pose=Transformation.From_Translation(np.array([0, 0, 0])),
)

# add the transmitter
base_station_device.transmitters.add(base_station_transmitter)

# =====================================
# Specify beamformer direction
# =====================================
# Focus spherical coordinates 0, 0, i.e. along the z-axis
beamformer.transmit_focus = SphericalFocus(0, 0)
_ = uniform_array.plot_pattern(carrier_frequency, beamformer)
plt.show()

## Prepare User Equipment

Create a User Equipment Receiver at (50, 50, 100)

Then set the focus of the beamformer transmitter to point to the UE (on focus) versus (off focus).  The received power is higher when the focus is towards the UE.


In [ ]:
from hermespy.core import SignalReceiver

# Create a new simulated device representing user equipment
user_equipment_device = simulation.new_device(
    carrier_frequency=carrier_frequency,
    pose=Transformation.From_Translation(np.array([50., 50., 100.])),
)

# Configure the user equipment to simply record 120 samples of any signal impinging on it
user_equipment_receiver = SignalReceiver(120, sampling_rate, sinusoid.power[0])
user_equipment_device.receivers.add(user_equipment_receiver)

In [ ]:
from hermespy.simulation import DeviceFocus

# =====================================
# Beam form towards UE device
# =====================================
# Focus the base station's main lobe on the user equipment
beamformer.transmit_focus = DeviceFocus(user_equipment_device)

# Visualize the beam pattern
_ = uniform_array.plot_pattern(carrier_frequency, beamformer)
off_target_focus = SphericalFocus(-.75 * np.pi, .3 * np.pi)
on_target_focus = DeviceFocus(user_equipment_device)
plt.show()

# =====================================
# Beamform towards Off-focuse device
# =====================================
beamformer.transmit_focus = off_target_focus
off_focus_drop = simulation.scenario.drop()
off_focus_drop.device_receptions[1].baseband_signal.plot(title='Off Target Reception').show()

beamformer.transmit_focus = on_target_focus
on_focus_drop = simulation.scenario.drop()
on_focus_drop.device_receptions[1].baseband_signal.plot(title='On Target Reception').show()

# Visualize the array's beamforming pattern when focusing the user equipment
beamformer.transmit_focus = on_target_focus
# hybrid_array.plot_pattern(carrier_frequency, beamformer)
plt.show()